In [1]:
import glob
import numpy as np
from model.srcnn import SRCNN
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import sys
from torchvision.utils import save_image, make_grid
from torchvision.models import vgg19

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from model.dataset import ImageDataset
from matplotlib import pyplot as plt


class Setup:
    def __init__(self):
        self.epoch = 0
        self.n_epochs= 200
        self.dataset_name = "ksdakf"
        self.batch_size = 8
        self.lr = 0.0002
        self.b1 = 0.5
        self.b2 = 0.999
        self.decay_epoch = 100
        self.n_cpu = 8
        self.hr_height = 256
        self.hr_width = 256
        self.channels = 3
        self.sample_interval = 100
        self.checkpoint_interval = 100
        self.training_image_dir  = 'data/train'
        self.validation_image_dir = 'data/validation'
        self.model_checkpoint_dir = 'model/checkpoints/SRCNN'
        self.results = 'data/results/SRCNN'
        self.validation_batch_size = 8
        


opt = Setup()

train_set = ImageDataset(opt.training_image_dir,(opt.hr_height,opt.hr_width),2)

train_loader = DataLoader(
    train_set,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=0)

validation_set = ImageDataset(opt.validation_image_dir,(opt.hr_height,opt.hr_width),2)

validation_loader = DataLoader(
    validation_set,
    batch_size=opt.validation_batch_size,
    shuffle=False,
    num_workers=0)

net = SRCNN(3)


c:\Users\BREAL\miniconda3\envs\ml\lib\site-packages\torchvision\transforms\transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [2]:
net.train_model(train_loader,validation_loader,opt)




plt.plot(net.train_psnr_log)
plt.plot(net.valid_psnr_log)
plt.title("PSNR plots")
plt.ylabel("PSNR")
plt.xlabel("epoch")
plt.legend(
    ["train", "validation"],
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0,
    fontsize="x-small",
)

plt.savefig(opt.results + "/SRCNN_Model_PSNR.png", bbox_inches="tight", dpi=600)
plt.show()

plt.plot(net.train_loss_log)
plt.plot(net.valid_loss_log)
plt.title("Loss plots")
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.legend(
    ["train", "validation"],
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0,
    fontsize="x-small",
)

plt.savefig(opt.results + "/SRCNN_Model_MSE_loss.png", bbox_inches="tight", dpi=600)
plt.show()

[Epoch 0/200] [Batch 0/100] [Loss: 1.283809]
[Epoch 0/200] [Batch 1/100] [Loss: 1.174046]


KeyboardInterrupt: 